# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.11it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.13it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.65it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.36it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.34it/s]



...

 78%|███████▊  | 18/23 [00:05<00:01,  4.68it/s]

 83%|████████▎ | 19/23 [00:05<00:00,  4.52it/s]

 91%|█████████▏| 21/23 [00:05<00:00,  4.73it/s]

 96%|█████████▌| 22/23 [00:06<00:00,  4.73it/s]

100%|██████████| 23/23 [00:06<00:00,  3.58it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Rachel. I am a 29-year-old teacher and a mother of two young children. I am writing this letter to my future self, hoping to capture the essence of this moment in time.
As I sit here in my cozy home, surrounded by the chaos of motherhood, I am filled with a sense of gratitude and wonder. My children, Emily and Jackson, are growing up so fast. They are at that magical age where everything is new and exciting, and they are constantly exploring the world around them.
As a teacher, I have the privilege of being a part of my students' educational journey, guiding them through the ups and
Prompt: The president of the United States is
Generated text:  the head of state and head of government of the United States, and is the highest-ranking official in the federal government. The president is also the commander-in-chief of the armed forces and has significant executive power. The president is elected by the Electoral College and serves a four-year ter

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Je

annie

 and

 I

...

.

 This

 is

 happening

 through

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Joseph and I'm a 17 year old student at a private boarding school in New York. I'm currently a junior and I'm struggling with keeping up with schoolwork and extracurricular activities. I'm not sure if I should take a gap year or stick with the traditional 4-year college route. I'm worried that if I take a gap year, I'll fall behind and not be as competitive when I apply to colleges. But, I'm also worried that if I stick with the traditional route, I'll burn out and not be able to fully enjoy my last year of high school.
It sounds like you're feeling

Prompt: The capital of France is
Generated text:  a city that has been shrouded in history, culture, and romance for centuries. Paris, the City of Light, is a place that has been a source of inspiration for artists, writers, and musicians for centuries. From the Eiffel Tower to the Louvre, there are countless iconic landmarks and attractions that make Paris a must-visit destination for anyone who

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 David

 and

 I

'm

...

 more

 likely

 to

 trust

 the

In [6]:
llm.shutdown()